# Semantiline Kernel koos OpenBnB MCP serveri integratsiooniga

See märkmik näitab, kuidas kasutada Semantilist Kernelit koos reaalse OpenBnB MCP serveriga Airbnb majutuste otsimiseks, kasutades MCPStdioPluginit. LLM-ile juurdepääsuks kasutatakse Microsoft Foundryt. Keskkonnamuutujate seadistamiseks võite järgida [Seadistuste õppetundi](/00-course-setup/README.md)


## Impordi vajalikud paketid


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP pistikprogrammi ühenduse loomine

Ühendame end [OpenBnB MCP serveriga](https://github.com/openbnb-org/mcp-server-airbnb) kasutades MCPStdioPlugin. See server pakub Airbnb otsingufunktsionaalsust läbi @openbnb/mcp-server-airbnb paketi.


## Kliendi loomine

Selles näites kasutame Microsoft Foundryt oma LLM-i juurdepääsu jaoks. Veenduge, et teie keskkonnamuutujad on õigesti seadistatud.


## Keskkonna seadistamine

Seadista Azure OpenAI sätted. Veendu, et sul on järgmised keskkonnamuutujad määratud:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP integratsiooni mõistmine

See märkmik ühendub **reaalse OpenBnB MCP serveriga**, mis pakub tegelikku Airbnb otsingufunktsionaalsust.

### Kuidas see töötab:

1. **MCPStdioPlugin**: Kasutab MCP serveriga suhtlemiseks standardset sisend/väljund kommunikatsiooni
2. **Tegelik NPM pakett**: Laeb alla ja käivitab `@openbnb/mcp-server-airbnb` npx kaudu
3. **Reaalajas andmed**: Tagastab Airbnb omaduste andmed otse nende API-dest
4. **Funktsioonide avastamine**: Agent avastab automaatselt MCP serverist saadaolevad funktsioonid

### Saadaval olevad funktsioonid:

OpenBnB MCP server pakub tavaliselt:
- **search_listings** - Otsi Airbnb omadusi asukoha ja kriteeriumite alusel
- **get_listing_details** - Hangi üksikasjalik info konkreetsete omaduste kohta
- **check_availability** - Kontrolli saadavust kindlate kuupäevade jaoks
- **get_reviews** - Hangi ülevaated omaduste kohta
- **get_host_info** - Hangi info omaduste majutajate kohta

### Eeltingimused:

- Süsteemis peab olema paigaldatud **Node.js**
- MCP serveri paketi allalaadimiseks on vaja **internetühendust**
- Kättesaadav peab olema **NPX** (tuleb Node.js-iga)

### Ühenduse testimine:

Saad MCP serverit käsitsi testida, käivitades:
```bash
npx -y @openbnb/mcp-server-airbnb
```

See laadib alla ja käivitab OpenBnB MCP serveri, millega Semantic Kernel seejärel ühendub, et saada tegelikke Airbnb andmeid.


## Agendi käivitamine OpenBnB MCP serveriga

Nüüd käivitame AI agendi, mis ühendub OpenBnB MCP serveriga, et leida päris Airbnb majutusi Stockholmis 2 täiskasvanu ja 1 lapse jaoks. Võite julgelt muuta `user_inputs` loendit, et muuta otsingukriteeriume.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Kokkuvõte
Palju õnne! Olete edukalt loonud tehisintellekti agendi, mis integreerub pärismaailma majutuspaikade otsinguga, kasutades Model Context Protocol’i (MCP):

## Kasutatud tehnoloogiad:
- Semantic Kernel – nutikate agentide loomine Azure OpenAI-ga
- Microsoft Foundry – LLM võimekused ja vestluse täitmine
- MCP (Model Context Protocol) – standardiseeritud tööriistade integreerimine
- OpenBnB MCP Server – reaalse Airbnb otsingu funktsionaalsus
- Node.js/NPX – välise MCP serveri käivitamine

## Mida te õppisite:
- MCP integratsioon: Semantic Kernel agentide ühendamine väliste MCP serveritega
- Reaalajas andmete juurdepääs: päris Airbnb kinnisvara otsimine otse API-de kaudu
- Protokolli suhtlus: stdio suhtluse kasutamine agendi ja MCP serveri vahel
- Funktsioonide avastamine: saadaolevate funktsioonide automaatne leidmine MCP serveritest
- Vooguvas vastuste vastuvõtt: funktsioonikõnede reaalajas püüdmine ja logimine
- HTML kuvamine: agendi vastuste vormindamine stiilsete tabelite ja interaktiivsete kuvadega

## Järgmised sammud:
- Täiendavate MCP serverite integreerimine (ilmajaam, lennud, restoranid)
- Mitme-agendi süsteemi loomine MCP ja A2A protokollide kombineerimisel
- Kohandatud MCP serverite loomine oma andmeallikatele
- Püsiva vestluse mälu rakendamine sessioonide vahel
- Agendi juurutamine Azure Functions’i koos MCP serveri orkestreerimisega
- Kasutaja autentimise ja broneerimise võimaluste lisamine


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastutusest loobumine**:  
See dokument on tõlgitud tehisintellekti tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palun arvestage, et automatiseeritud tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument oma emakeeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tekkida võivate arusaamatuste või valesti mõistmiste eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
